In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta import *

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [29]:
df = spark.createDataFrame([[i, random.choice(string.ascii_uppercase)] for i in range(100)], ['id', 'value'])

In [30]:
additional_options = {
    "path": "/root/Work/Repositories/memory/training/pyspark/PracticeProblems/delta"
}
df.write.format('delta').saveAsTable("test_delta_table")

23/08/28 23:02:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/08/28 23:02:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/08/28 23:02:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/08/28 23:02:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/08/28 23:02:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/08/28 23:02:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/08/28 23:02:40 WARN MemoryManager: Total allocation exceeds 95.

In [31]:
spark.sql('select * from test_delta_table').show()

+---+-----+
| id|value|
+---+-----+
|  0|    W|
|  1|    V|
|  2|    D|
|  3|    V|
|  4|    U|
|  5|    F|
| 66|    Z|
| 67|    W|
| 68|    H|
| 69|    A|
| 70|    I|
| 71|    B|
| 60|    P|
| 61|    V|
| 62|    O|
| 63|    C|
| 64|    S|
| 65|    N|
| 72|    D|
| 73|    G|
+---+-----+
only showing top 20 rows



In [32]:
spark.sql('select * from test_delta_table').count()

100

In [33]:
spark.sql('DELETE from test_delta_table where id = 1')

DataFrame[num_affected_rows: bigint]

In [34]:
spark.sql('select * from test_delta_table').count()

99

In [35]:
spark.sql('OPTIMIZE test_delta_table ZORDER BY id')

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [37]:
spark.sql('DESCRIBE HISTORY test_delta_table').show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      2|2023-08-28 23:14:...|  null|    null|            OPTIMIZE|{predicate -> [],...|null|    null|     null|          1|SnapshotIsolation|        false|{numRemovedFiles ...|        null|Apache-Spark/3.4....|
|      1|2023-08-28 23:04:...|  null|    null|              DELETE|{predicate -> ["(...|null|    null|     null|          0|     Serializable|        fa

In [40]:
spark.sql('SELECT count(*) from test_delta_table VERSION AS OF 0').show()

+--------+
|count(1)|
+--------+
|     100|
+--------+



In [41]:
spark.sql('SELECT count(*) from test_delta_table VERSION AS OF 1').show()

+--------+
|count(1)|
+--------+
|      99|
+--------+



In [43]:
spark.sql('VACUUM test_delta_table').show()

Deleted 0 files and directories in a total of 1 directories.
+--------------------+
|                path|
+--------------------+
|file:/root/Work/R...|
+--------------------+

